In [ ]:
import os
import sqlite3
import MetaTrader5 as mt5
from datetime import datetime
import time
# from MT5 Python API documentation: 
# import pytz module for working with time zone
import pytz 
# set time zone to UTC
# create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
timezone = pytz.timezone('UTC')

%load_ext dotenv
%dotenv .env

def download_mt5_historical_quotes(symbols, timeframe, start, end, db_path):
    """
    Download MT5 historical quotes and store them in SQLite database.
    
    Args:
        symbols (list): List of MT5 symbols (e.g., ['GOOGL', 'NVDA']).
        timeframe: MT5 timeframe constant (e.g., mt5.TIMEFRAME_H1).
        start (str): Start date ('YYYY-MM-DD').
        end (str): End date ('YYYY-MM-DD').
        db_path (str): SQLite database file path.
    """
    # Initialize MT5 connection
    if not mt5.initialize():
        print("MT5 initialize() failed, error code =", mt5.last_error())
        return False
    
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Process each symbol
        for symbol in symbols:
            # Get or create symbol in the database and get its symbol_id
            symbol_id = get_or_create_symbol(cursor, symbol)
            print(f"symbol_id: {symbol_id}")
            
            if symbol_id is None:
                print(f"Failed to get/create symbol {symbol} in database")
                continue
            
            # Convert dates to datetime objects
            start_date = datetime.strptime(start, '%Y-%m-%d')
            end_date = datetime.strptime(end, '%Y-%m-%d')
            
            # from MT5 Python API documentation:
            # use UTC time zone 
            # to avoid the implementation of a local time zone offset
            utc_from = start_date.replace(tzinfo=timezone)  # Set timezone to UTC
            utc_to = end_date.replace(tzinfo=timezone)  # Set timezone to UTC

            # Convert to MT5's time format (POSIX timestamp)
            from_date = int(time.mktime(utc_from.timetuple()))
            to_date = int(time.mktime(utc_to.timetuple())) + 86400  # Add one day
            
            # Get historical data from MT5
            rates = mt5.copy_rates_range(symbol, timeframe, from_date, to_date)
            
            if rates is None:
                print(f"No data for {symbol}, error = {mt5.last_error()}")
                continue
            
            # Insert market data into database
            insert_market_data(cursor, symbol_id, timeframe, rates)
            
            print(f"Processed {len(rates)} records for {symbol}")
        
        # Commit changes
        conn.commit()
        return True
        
    except Exception as e:
        print(f"Error: {e}")
        return False
        
    finally:
        # Close connections
        if 'conn' in locals():
            conn.close()
        mt5.shutdown()

def get_or_create_symbol(cursor, mt5_symbol):
    """
    Get or create a symbol in the database and return its symbol_id.
    
    Args:
        cursor: SQLite cursor
        mt5_symbol: MT5 symbol string (e.g., 'GOOGL')
    
    Returns:
        symbol_id or None if failed
    """
    # Try to get existing symbol
    cursor.execute("SELECT symbol_id FROM Symbol WHERE ticker = ?", (mt5_symbol,))
    row = cursor.fetchone()
    
    if row:
        return row[0]
    
    # Get symbol info from MT5
    symbol_info = mt5.symbol_info(mt5_symbol)
    if symbol_info is None:
        print(f"Symbol info not found for {mt5_symbol}")
        return None
    
    # Insert new symbol
    cursor.execute("""
    INSERT INTO Symbol (ticker, exchange, asset_type, sector, industry, currency)
    VALUES (?, ?, ?, ?, ?, ?)
    """, (
        mt5_symbol,
        # some of this data will be filled by the MQL5 DB Update Service
        # because Python MT5 API does not provide some of them
        symbol_info.exchange or 'UNKNOWN',
        symbol_info.asset_class or 'UNKNOWN',
        symbol_info.sector or 'UNKNOWN',
        symbol_info.industry or 'UNKNOWN',
        symbol_info.currency_profit or 'UNKNOWN'
    ))
    
    return cursor.lastrowid

def insert_market_data(cursor, symbol_id, timeframe, rates):
    """
    Insert market data into the database.
    
    Args:
        cursor: SQLite cursor
        symbol_id: ID of the symbol in Symbol table
        timeframe: MT5 timeframe string
        rates: numpy array of market data from MT5
    """
    # Convert timeframe to string representation
    tf_str = timeframe_to_string(timeframe)
    
    # Prepare data for insertion
    data_to_insert = []
    for rate in rates:
        data_to_insert.append((
            int(rate['time']),
            tf_str,
            float(rate['open']),
            float(rate['high']),
            float(rate['low']),
            float(rate['close']),
            int(rate['tick_volume']),
            int(rate['real_volume'] if 'real_volume' in rate.dtype.names else 0),
            float(rate['spread'] if 'spread' in rate.dtype.names else 0),
            symbol_id
        ))
    
    # Insert data
    cursor.executemany("""
    INSERT OR IGNORE INTO market_data (
        tstamp, timeframe, price_open, price_high, price_low, price_close,
        tick_volume, real_volume, spread, symbol_id
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, data_to_insert)

def timeframe_to_string(timeframe):
    """Convert MT5 timeframe constant to string representation."""
    timeframes = {
        mt5.TIMEFRAME_M1: "M1",
        mt5.TIMEFRAME_M2: "M2",
        mt5.TIMEFRAME_M3: "M3",
        mt5.TIMEFRAME_M4: "M4",
        mt5.TIMEFRAME_M5: "M5",
        mt5.TIMEFRAME_M6: "M6",
        mt5.TIMEFRAME_M10: "M10",
        mt5.TIMEFRAME_M12: "M12",
        mt5.TIMEFRAME_M15: "M15",
        mt5.TIMEFRAME_M20: "M20",
        mt5.TIMEFRAME_M30: "M30",
        mt5.TIMEFRAME_H1: "H1",
        mt5.TIMEFRAME_H2: "H2",
        mt5.TIMEFRAME_H3: "H3",
        mt5.TIMEFRAME_H4: "H4",
        mt5.TIMEFRAME_H6: "H6",
        mt5.TIMEFRAME_H8: "H8",
        mt5.TIMEFRAME_H12: "H12",
        mt5.TIMEFRAME_D1: "D1",
        mt5.TIMEFRAME_W1: "W1",
        mt5.TIMEFRAME_MN1: "MN1"
    }
    return timeframes.get(timeframe, str(timeframe))


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
import MetaTrader5 as mt5

symbols = ['MPWR', 'AMAT', 'MU']  # Symbols from Market Watch
timeframe = mt5.TIMEFRAME_M5	# 5-minute timeframe
start_date = '2024-02-01'
end_date = '2024-03-31'
db_path = os.getenv('STATARB_DB_PATH')  # Path to your SQLite database

if db_path is None:
	print("Error: STATARB_DB_PATH environment variable is not set.")
else:
	print("db_path: " + db_path)
# Download historical quotes and store them in the database
	download_mt5_historical_quotes(symbols, timeframe, start_date, end_date, db_path)

db_path: C:\Users\gavra\AppData\Roaming\MetaQuotes\Terminal\73B7A2420D6397DFF9014A20F1201F97\MQL5\Files\StatArb\statarb-0.1.db
symbol_id: 7
Processed 2877 records for MPWR
symbol_id: 8
Processed 3025 records for AMAT
symbol_id: 9
Processed 3114 records for MU
